<!-- ## SMALL-SCALE CROP-MAPPING(Food security) -->

<center><h1>Flood Mapping Prediction System</h1> </center>


In [15]:
# #To install the libraries use !pip if any library is missing use !pip install name of library 
# !pip install geemap
# !pip install geopandas
# !pip install ipywidgets
# !pip install scikit-learn

In [16]:
#Importation of variAous Libraries or packages...
import os
from os import path as op
import geopandas as gdp
import geemap
import ee


# Making module or packages collection attribute became callable..
import collections
collections.Callable = collections.abc.Callable

#plotting packages
import pandas as pd#Used for data analysis also include DataFrame df data structure
import matplotlib.pyplot as plt#For plotting of dataframes
import numpy as np
# import altair as alt#declarative visualization(Used for charting)

import ipywidgets as widgets
# from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
from ipywidgets import HTML
from IPython.display import display, clear_output
from ipywidgets import AppLayout

from geemap import chart #for chart plotting


#packages used in database connection.ONLY applicable where on has the ability to connect to database contain data
import pandas.io.sql as sqlio
#Main function is to out various warning that might arise such as new upadates in package version
import warnings
warnings.filterwarnings("ignore")
Map=geemap.Map()
from geemap import Map
from ipywidgets import Button, Layout

map_layout= Map(layout=Layout(height='100px'))
map_layout.setCenter( 40.5, -2.5, 7)


map_layout.add_basemap(basemap='HYBRID')

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [17]:
import ipywidgets as widgets
from ipyleaflet import WidgetControl

# Style dictionary
style = {'description_width': 'initial'}

# General output widget
output_widget = widgets.Output(layout={'border': '1px solid red', 'align-items': 'center', 'background-color': 'red'})

# Widget used to display crop health
output_widget_health = widgets.Output(layout={'border': '1px solid black', 'marginright': '150px'})

# General output control tool.
output_control = WidgetControl(widget=output_widget, position='topright')
# Map output control
map_layout.add_control(output_control)

# AOI Area of Interest Selection of the study area.
AOI = widgets.Button(
    description='Drawn Study Area',
    button_style='success',
    tooltip='Draw your boundaries',
    style=style
)

# Create a date picker widget for start date
start_date_picker = widgets.DatePicker(
    description='Select Start Date:',
    disabled=False
)

# Create a date picker widget for the end date
end_date_picker = widgets.DatePicker(
    description='Select End Date:',
    disabled=False
)

# Create a button to apply the date filter
apply_date_filter_button = widgets.Button(
    description='Apply Date Filter',
    disabled=False,
    button_style='success',
    style=style
)

Image_Preprocessing = widgets.Button(
    description='Image Preprocessing',
    button_style='info',
    tooltip='Image Preprocessing process',
    style=style
)

Update_Mask = widgets.Button(
    description='Process Variables',
    button_style='danger',
    tooltip='Passing input variables for prediction purposes',
    style=style
)

Buildings = widgets.Button(
    description='Population Response',
    button_style='primary',
    tooltip='Load the machine learned model',
    style=style
)

Export_data = widgets.Button(
    description='Calculate Mean',
    button_style='success',
    tooltip='Generate Mean Value',
    style=style
)

Flood_Flow = widgets.Button(
    description='Flood Flow Prediction',
    button_style='warning',
    tooltip='Estimated Flow',
    style=style
)

Reset_button = widgets.Button(
    description='Reset',
    button_style='danger',
    tooltip='Reset using this button',
    style=style
)

Data_Input = widgets.Button(
    description='Estimate Biomass Value',
    button_style='info',
    tooltip='Estimate Biomass Value',
    style=style
)

cal_area = widgets.Button(
    description='Estimate Flood Area',
    button_style='info',
    tooltip='Estimate Flood Area',
    style=style
)

ML_learned_model = widgets.Button(
    description='ML Water Level Predictor',
    button_style='primary',
    tooltip='Use a trained model to predict possible water rise',
    style=style
)

LULCC = widgets.Button(
    description='Carry-Out LULC',
    button_style='info',
    tooltip='Dynamic Worlds',
    style=style
)

check_box_sent_image = widgets.Checkbox(
    value=False,
    description='Download Results',
    style=style
)

check_box_classification = widgets.Checkbox(
    value=False,
    description='Download LULC Results',
    style=style
)

check_box_flood_map = widgets.Checkbox(
    value=False,
    description='Download Mapped Results',
    style=style
)

Export_data = widgets.Button(
    description='Download Results',
    button_style='info',
    tooltip='Download results of the selected feature i.e classification results',
    style=style
)

lbl1 = widgets.Label('Rainfall')
text1 = widgets.Text(
    description='',
    value='Input Rainfall',
    width=100,
    style=style
)

lbl2 = widgets.Label('Temperature')
text2 = widgets.Text(
    description='',
    value='Input Temperature',
    width=100,
    style=style
)

text3 = widgets.Text(
    description='',
    value='Input Humidity',
    width=100,
    style=style
)

text4 = widgets.Text(
    description='',
    value='Input Evaporation',
    width=100,
    style=style
)

btn = widgets.Button(
    description='Submit Inputs',
    button_style='primary',
    tooltip='Submit above inputs',
    style=style
)

lbl3 = widgets.Label()


In [18]:
output_widget88 = widgets.Output(layout={'border': '1px solid black'})
output_control88 = WidgetControl(widget=output_widget88, position='bottomleft')
map_layout.add_control(output_control88)

In [19]:
style = {'description_width': 'initial'}
uploader = widgets.FileUpload(
    description='Upload data',
    accept='.zip, .json, .geojson',
    multiple=False,
    button_style='primary',
    style=style,
)

submit = widgets.Button(
    description='Display data', button_style='success', tooltip='Click me', style=style
)

reset = widgets.Button(
    description='Reset', button_style='warning', tooltip='Click me', style=style
)

In [20]:
with output_widget88:
    print('Upload shapefile or \ngeojson as a zip file')
    display(uploader)
    display(submit)
    display(reset)

In [21]:
def get_vector(upload_widget, out_dir=None):

    import zipfile
    import glob

    if out_dir is None:
        out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    vector = None

    try:
        [uploaded_file] = upload_widget.value
        file = upload_widget.value[uploaded_file]
        name = file['metadata']['name']
        content = file['content']
        out_file = os.path.join(out_dir, name)
        with open(out_file, "wb") as fp:
            fp.write(content)

        if name.endswith('.zip'):
            with zipfile.ZipFile(out_file, "r") as zip_ref:
                extract_dir = os.path.join(
                    out_dir, name[:-4] + "_" + geemap.random_string(3)
                )
                zip_ref.extractall(extract_dir)
                files = glob.glob(extract_dir + '/*.shp')
                if len(files) > 0:
                    shp = files[0]
                    vector = geemap.shp_to_ee(shp)
                else:
                    files = glob.glob(extract_dir + '/*.geojson')
                    if len(files) > 0:
                        geojson = files[0]
                        vector = geemap.geojson_to_ee(geojson)
        else:
            vector = geemap.geojson_to_ee(out_file)

    except Exception as e:
        print(e)

    return vector

In [22]:
#Trial to avoid printing error in the code but a message to the user
def AOI_clicked(b):
        with output_widget:
            output_widget.clear_output()
            

import os

print(os.listdir("D:\PROJECT\LowerTana Data\ACTUAL TRAINING DATA"))  # Name of the project Directory
os.getcwd()
# Read rainfall data
            
rainfall=pd.read_csv("D:\PROJECT\LowerTana Data\ACTUAL TRAINING DATA\Rainfall2.csv")

#Read waterlevel readings
waterlevel=pd.read_csv("D:\PROJECT\LowerTana Data\ACTUAL TRAINING DATA\Water_Level.csv")
            
#Creaton of a list to hold both the two datasets from different dataframes 
values_list=[rainfall,waterlevel]
#Merging the two datasets into a single dataframe.
variable_concat=pd.concat(values_list,axis=1)
#Dfine X and Y variables
#iloc function to select a rows or alt select based on column name..
#Assigning the selected rows axis eg x and y axis.
x_variable = variable_concat[["precipitation"]].values
y_variable = variable_concat[["water_level"]].values
#splitting data
#80% training and 20% validation
#Aspect of Random With random_state=None , we get different train and test sets across different executions and the shuffling
# process is out of control. With random_state=0 , we get the same train and test sets across different executions.

x_train_climatic, x_test_climatic,y_train_climatic, y_test_climatic= train_test_split(x_variable,y_variable,test_size=0.1,random_state=0)


# Based on Regression analysis is a predictive modeling technique that estimates the relationship between two or more variables. 
# statistical way of measuring the relationship between one or more independent variables vs one dependent variable. 

               
from sklearn.linear_model import LinearRegression
# statistical way of measuring the relationship between one or more independent variables vs one dependent variable. 
global rain_level_regressor
rain_level_regressor = LinearRegression()
rain_level_regressor.fit(x_train_climatic,y_train_climatic)
y_pred_water_level= rain_level_regressor.predict(x_test_climatic)

#Random forest model.
from sklearn.ensemble import RandomForestRegressor

Random_regressor = RandomForestRegressor(n_estimators =10, random_state = 100)
# Train the model on training data
Random_regressor.fit(x_train_climatic,y_train_climatic)

# <<<<End of Random Forest model analysis>>>>
User_Rainfall_Amount_new=(np.array(rainfall).reshape(1,-1))
global Rainfall_Input
Rainfall_Input=User_Rainfall_Amount_new

predicted_water_level_input=rain_level_regressor.predict(rainfall)
#Conversion of a 2D to 3D----Reshaping
x=predicted_water_level_input[0]
predicted_water_level_input=x[0]
# Round off to-4 figure
global predicted_waterlevel_value88
predicted_waterlevel_value88=(round(predicted_water_level_input, 4))
print("Expected water rise will be:",predicted_waterlevel_value88)
           
                
ML_learned_model.on_click(AOI_clicked)

['20231103045855-27171-data.txt', 'doc.kml', 'Humidity.csv', 'LowerTana.kmz', 'LowerTana2.json', 'LowerTana2.kmz', 'mygeodata', 'occurence.csv', 'Population.csv', 'Rainfall.csv', 'Rainfall2.csv', 'Rainfall3.csv', 'Soil moisture.csv', 'Soil moisture2.csv', 'Temperature.csv', 'Temperature2.csv', 'Water_Level.csv', 'Water_Level2.csv']


ValueError: could not convert string to float: '{"type":"MultiPoint","coordinates":[]}'

In [23]:
def submit_clicked(b):
    if uploader._counter > 0:
        map_layout.default_style = {'cursor': 'wait'}
        try:
            fc = get_vector(uploader)
            layer_name = 'Layer ' + geemap.random_string(3)
            map_layout.addLayer(fc, {}, layer_name)
            map_layout.centerObject(fc)
            uploader.value.clear()
            uploader._counter = 0
        except Exception as e:
            print(e)
        map_layout.default_style = {'cursor': 'pointer'}


submit.on_click(submit_clicked)


def reset_clicked(b):

    map_layout.layers = map_layout.layers[:3]
    output_widget88.clear_output()
    with output_widget88:
        print('Upload shapefile or \ngeojson as a zip file')
        display(uploader)
        display(submit)
        display(reset)
    uploader.value.clear()
    uploader._counter = 0


reset.on_click(reset_clicked)

In [24]:
#User instructions manual.
instructions=widgets.Dropdown(
    options=['Please follow this steps sequentially',
             '1.Drawn study area or upload',
             '2.Input the variables which will be used for prediction purpose,Then Submt results',
             '3.Carry out Image Processing based on sentinel Data',
             '4.Alternatively use Dynamic World for image classification(if you want to work with another study area)',
             '5.Used trained Machine Lerning Model to predict the level of water level',
             '6.Process the variables:passing  through the input variables to a trained refined lee method',
             '7.Flood Flow: Estimation the possible flood extend',
             '8. Calculate the possible flood area extended',
             '9.Estimate the Population likely to be affected by flood',
             '10.Reset or clear everything'],
    value='Please follow this steps sequentially',
    description='Instruction:',
    disabled=False,
)

In [25]:
#Reset everything
def add(b):
    with output_widget:
        output_widget.clear_output()
        try:
            global rainfall
            rainfall=int(text1.value)
            global y
            y=int(text2.value)
            global humidity
            humidity=int(text3.value)
            global evaporation
            evaporation=int(text4.value)
           
        except Exception as e:
            print("Sorry an error just occurred...Please fill all the required inputs..please")
        else:
            print("Successfully submitted the inputs....")

btn.on_click(add)

In [26]:

# def add(b):
#     global rainfall
#     rainfall=int(text1.value)
#     global y
#     y=int(text2.value)
#     global humidity 
#     humidity=int(text3.value)
#     global evaporation 
#     evaporation=int(text4.value)

# btn.on_click(add)

In [27]:
pip install voila


Note: you may need to restart the kernel to use updated packages.


In [28]:
#Arrange the layout
from ipywidgets import AppLayout
verticalBox = output_widget
vBox1 = widgets.VBox([map_layout])


vBox2 = widgets.VBox([instructions,AOI,start_date_picker, end_date_picker, apply_date_filter_button,
                      text1,text2,text3,text4,btn,Image_Preprocessing,LULCC,ML_learned_model,
                      Update_Mask,Flood_Flow,cal_area,Buildings,check_box_sent_image,
                     Export_data,Reset_button])
# check_box_classification,check_box_flood_map

AppLayout(header=None,
          left_sidebar=vBox2,
          right_sidebar=vBox1,
          footer=None,
          step= 1,
          pane_widths=[1, 1, 3],
          pane_heights=[1, 1, '500%'])

AppLayout(children=(VBox(children=(Dropdown(description='Instruction:', options=('Please follow this steps seq…

In [29]:
#Trial the associated function below is to enable on to draw  their region  and save it as feature collection 
def AOI_clicked(b):
        with output_widget:
#             output_widget.clear_output()
           
            try:
                global boundaries
                boundaries=map_layout.user_rois
                map_layout.addLayer(boundaries,{},"Region of Intrest")
            
         
            except Exception as e:
                print('Please select Your area of Interest.....')
            else:
                print("Successfully drawn your Region of Interest ")
                output_widget.clear_output()
        
AOI.on_click(AOI_clicked)

In [30]:
# Click event handler for Image processing process e.g sentinel data
def preprocess_img_clicked(b):
     with output_widget:
            output_widget.clear_output()
            try:
                start_date="2020-01-01"#Set start_date(yy/mon/day)
                end_date="2020-03-31"#Set End_date(yy/mon/day)
                season = ee.Filter.date(start_date,end_date);#Filter image based on the time frame(start_date and end_date
                try:
                    sentinel_1= ee.ImageCollection('COPERNICUS/S1_GRD');
                    sCollection=sentinel_1\
                    .filterBounds(boundaries) \
                    .filter(season) \
                    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
                    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
                    .filter(ee.Filter.eq('instrumentMode', 'IW'))
              
                    desc=sCollection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'));
                    asc=sCollection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'));

                    global composite
                    composite = ee.Image.cat([
                asc.select('VH').mean(),
                asc.select('VV').mean(),
                desc.select('VH').mean()
                ]).focal_median();
   
 #-------------SENTINEL_2A DATA----------------------#  
                    global sentinel_2A
                    sentinel_2A = ee.ImageCollection('COPERNICUS/S2')\
               .filterBounds(boundaries)\
               .filterDate("2020-01-01","2020-03-31")\
               .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",10))\
               .median()\
               .select('B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8', 'B10', 'B11')\
               .clip(boundaries)
                except Exception as e:
                    print("***Hint this may be caused by missing ROI.....please draw your ROI first***")
#                

                sentinel_2Avispar={"min":0, "max":2500,"bands": ['B4','B3','B2']}#Visualization parameters used.

                map_layout.addLayer(sentinel_2A,sentinel_2Avispar, 'Sentinel-2')#Add sentinel_2A to the layer 
        
            except Exception as e:
                print('Sorry an error occurred...please try again')
            else:
                print("Image preprocessing attained successfully")
        
Image_Preprocessing.on_click(preprocess_img_clicked)

In [31]:
#Image classification from sentinel 2A and Sentinel 1A
def Buildings_clicked(b):
    with output_widget:
        output_widget.clear_output()
        
        buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v1/polygons');
        roi_buildings=buildings\
        .filter('confidence >= 0.70')\
        .filterBounds(boundaries)\
        
        buildings_vispar=['00FF00']
        map_layout.addLayer(roi_buildings,{'color': 'ff0000'},"building distribution")
        
        buffer=roi_buildings.map(lambda f: f.buffer(30))
        map_layout.addLayer(buffer,{'color': 'FFAE00'},"30M_Buffered",False)
        
        #Creation of heat map

        def func_tbb(buildinds_feature_collection):
            return buildinds_feature_collection.set('google_buildinds',1)

        google_buildings = roi_buildings.map(func_tbb)

        def heatmap(google_buildings,radius):
            ptImg = google_buildings.reduceToImage(['google_buildinds'],ee.Reducer.first()).unmask(0)
            kernel = ee.Kernel.circle(radius)
            result = ptImg.convolve(kernel)
            return result.updateMask(result.neq(0))

# Adjust the buffering radius
        Buildings_heatmap = heatmap(google_buildings,8)
#Colour Gradient
        gradient = ['green','yellow','red']
        map_layout.addLayer(Buildings_heatmap,{'palette':gradient, 'min':0, 'max':0.6},'Built-up Regions')


Buildings.on_click(Buildings_clicked)

In [32]:
# Click event handler for Dynamic world classification
def submit_clicked(b):
     with output_widget:
            try:
                output_widget.clear_output()
                map_layout.remove_legends()
                start_date="2018-01-01"#Set start_date(yy/mon/day)
                end_date="2022-03-31"#Set End_date(yy/mon/day)
            
                global region # Set the region of interest by simply drawing a polygon on the map
                region = boundaries
                if region is None:
                    region = ee.Geometry.BBox(-61.18116254022864,6.614609265030635, -61.18116254022864,6.769074870340775)

                image = geemap.dynamic_world_s2(region, start_date, end_date)
                vis_params3 = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
                global landcover
                # Create Dynamic World land cover composite
                landcover = geemap.dynamic_world(region, start_date, end_date, return_type='hillshade')

                # Add legend to the map
                map_layout.addLayer(landcover.clip(boundaries),{}, 'LULCC') 
                map_layout.add_legend(title="Land Cover", builtin_legend='Dynamic_World')
                

            except Exception as e:
                print('Sorry an error occurred...please try again')
            else:
                print("LULC attained successfully")
        
LULCC.on_click(submit_clicked)

In [33]:
#Crop masking: This is to isolate other classes aside from cropland
def AOI_clicked(b):
        with output_widget:
            
            output_widget.clear_output()
            
            blue_palette = ['0000FF']
            jrcGSW = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
            waterBodies = jrcGSW.select('occurrence').clip(boundaries)
            
            vis_waterBodies = {min:0, max:100, "palette": blue_palette}
            map_layout.addLayer(**{
    # mask waterBodies so as to not detect flood in the waterBodies.
    # .divide(100) causes the opacity/transparency of the pixels to
    # be set based on the waterBodies occurrence value.
    
    'ee_object': waterBodies.updateMask(waterBodies.divide(100)),\
    'name': "Permanent water bodies (BLUE)", \
    'vis_params': vis_waterBodies })
#             map_layout.addLayer( waterBodies.updateMask(waterBodies.divide(100)),vis_waterBodies,"Permanent water bodies (BLUE)")
            
            empty = ee.Image().byte()
            slope_palette88 = ['ff0000']
            filledOutlines = empty.paint(boundaries, 'BIOME_NUM').paint(boundaries, 0, 2)
            
#             map_layout.addLayer(filledOutlines, {"palette": slope_palette88, max: 14}, 'ROI')
            
            #Set the time stand
            beforeStart = '2021-03-13'
            beforeEnd = '2021-06-13'

            afterStart = '2021-08-21'
            afterEnd = '2021-09-16'
            
            s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
            s1Collection = s1.filter(ee.Filter.eq('instrumentMode', 'IW'))\
                      .filter(ee.Filter.eq('transmitterReceiverPolarisation', ['VV','VH'])) \
                      .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
                      .filter(ee.Filter.eq('resolution_meters',10)) \
                      .filterBounds(boundaries) \
                      .select('V.*')
            global before_collection 
            before_collection = s1Collection.select('VH').filterDate(beforeStart, beforeEnd)
            global after_collection 
            after_collection = s1Collection.select('VH').filterDate(afterStart, afterEnd)
            global before
            before = before_collection.mosaic().clip(boundaries)
            global after
            after = after_collection.mosaic().clip(boundaries)
            
            
            # Function to convert from dB
            global toNatural
            def toNatural(img):
                return ee.Image(10.0).pow(img.select(0).divide(10.0))

# Function to convert to dB
            global toDB
            def toDB(img):
                return ee.Image(img).log10().multiply(10.0)
            global RefinedLee
            def RefinedLee(img):
            
                weights3 = ee.List.repeat(ee.List.repeat(1,3),3)
                kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False)
    
                mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
                variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

    # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
                sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],\
                             [0,0,0,0,0,0,0], [0,1,0,1,0,1,0], \
                             [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]])
        
                sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False)

    # Calculate mean and variance for the sampled windows and store as 9 bands
                sample_mean = mean3.neighborhoodToBands(sample_kernel)
                sample_var = variance3.neighborhoodToBands(sample_kernel)

    # Determine the 4 gradients for the sampled windows
                gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
                gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
                gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
                gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

    # And find the maximum gradient amongst gradient bands
                max_gradient = gradients.reduce(ee.Reducer.max())

    # Create a mask for band pixels that are the maximum gradient
                gradmask = gradients.eq(max_gradient)

    # duplicate gradmask bands: each gradient represents 2 directions
                gradmask = gradmask.addBands(gradmask)

    # Determine the 8 directions
                directions = sample_mean.select(1) \
                            .subtract(sample_mean.select(4)) \
                            .gt(sample_mean.select(4).subtract(sample_mean.select(7))) \
                            .multiply(1)
                directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)) \
                           .gt(sample_mean.select(4).subtract(sample_mean.select(2))) \
                           .multiply(2))
                directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)) \
                           .gt(sample_mean.select(4).subtract(sample_mean.select(5))) \
                           .multiply(3))
                directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)) \
                           .gt(sample_mean.select(4).subtract(sample_mean.select(8))) \
                           .multiply(4))
    # The next 4 are the not() of the previous 4
                directions = directions.addBands(directions.select(0).Not().multiply(5))
                directions = directions.addBands(directions.select(1).Not().multiply(6))
                directions = directions.addBands(directions.select(2).Not().multiply(7))
                directions = directions.addBands(directions.select(3).Not().multiply(8))

    # Mask all values that are not 1-8
                directions = directions.updateMask(gradmask)

    # "collapse" the stack into a singe band image (due to masking, each pixel has just 
    # one value (1-8) in it's directional band, and is otherwise masked)
                directions = directions.reduce(ee.Reducer.sum()) 

                pal = ['ffffff','ff0000','ffff00', '00ff00', '00ffff', '0000ff', 'ff00ff', '000000']
#     Map.addLayer(directions.reduce(ee.Reducer.sum()), {min:1, max:8, palette: pal}, 'Directions', false)

                sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

    # Calculate localNoiseVariance
                sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0])

    # Set up the 7*7 kernels for directional statistics
                rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4))

                diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0],\
                                [1,1,1,0,0,0,0], [1,1,1,1,0,0,0], \
                                [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]])

                rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False)
                diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False)

    # Create stacks for mean and variance using the original kernels. Mask with relevant direction.
                dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
                dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

                dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
                dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

    # and add the bands for rotated kernels
    #for  (var i=1; i<4; i++):
       
                for i in range(1, 4):
                        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)) \
                                        .updateMask(directions.eq(2*i+1)))
                        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)) \
                                        .updateMask(directions.eq(2*i+1)))
                        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)) \
                                        .updateMask(directions.eq(2*i+2)))
                        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)) \
                                        .updateMask(directions.eq(2*i+2)))  

    # "collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
                        dir_mean = dir_mean.reduce(ee.Reducer.sum())
                        dir_var = dir_var.reduce(ee.Reducer.sum())

    # A finally generate the filtered value
                        varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))

                        b = varX.divide(dir_var)

                        result = dir_mean.add(b.multiply(img.subtract(dir_mean)))
                return(result.arrayFlatten([['sum']]))
        before_rLee = ee.Image(toDB(RefinedLee(toNatural(before))))
        after_rLee = ee.Image(toDB(RefinedLee(toNatural(after))))
           
        difference_rLee = after_rLee.divide(before_rLee)
        
        DIFF_UPPER_THRESHOLD = 1.25

# Identify pixels above the threshold (i.e flood pixels).
# Set all other pixels to 0
        floodMask = difference_rLee.gt(DIFF_UPPER_THRESHOLD).rename('water') 

# Keep only the flood pixels. Remove all pixels equal to 0
        floodedAreas = floodMask.selfMask()
        jrcGSW = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
        waterBodies = jrcGSW.select('occurrence').clip(boundaries)
        vis_waterBodies = {min:0, max:100, "palette": slope_palette88}
          
        map_layout.addLayer(**{
    # mask waterBodies so as to not detect flood in the waterBodies.
    # .divide(100) causes the opacity/transparency of the pixels to
    # be set based on the waterBodies occurrence value.
    
    'ee_object': waterBodies.updateMask(waterBodies.divide(100)),\
    'name': "Permanent water bodies (BLUE)", \
    'vis_params': vis_waterBodies })
        
        seasonality = jrcGSW.select('seasonality')
        
        permWater = seasonality.gte(10).selfMask()
        
        permWater_removed = floodedAreas.where(permWater,0)
  
# Remove all the permanent water present in the flooded layer.
# floodedAreas layer has only flood pixels
        onlyFlooded = floodedAreas.updateMask(permWater_removed).selfMask()
 
# Set minimum flood area in pixels = 3, approximately 0.25 ha
        minFloodPixels = ee.Number(3)
  
# Scale the results to display on the map. 
# Reprojecting with a specified scale ensures that the pixel area 
# does not change with zoom.
        prj = jrcGSW.projection()
        scale = prj.nominalScale()
        contFloodArea = onlyFlooded.connectedPixelCount(25) \
                      .reproject(prj.atScale(scale))
                      
# Apply the minimum area requirement.
        minFloodArea = contFloodArea.gte(minFloodPixels).selfMask() \
                    .reproject(prj.atScale(scale))
        demHydroSHEDS = ee.Image("WWF/HydroSHEDS/03VFDEM")
        terrain = ee.Algorithms.Terrain(demHydroSHEDS)
        slope = terrain.select('slope')
        
        finalFloodedArea = minFloodArea.updateMask(slope.lt(5))
        slope_palette1 = ['0C7600','4CE500','B2FF18','FFFF00','FFAE00','ff6d66','FF0000']
#         map_layout.addLayer(finalFloodedArea,{'palette':slope_palette1},"keserian_slope")
        global after_filter
        after_filter = ee.Image(toDB(RefinedLee(toNatural(after))))
        global before_filter
        before_filter = ee.Image(toDB(RefinedLee(toNatural(before))))
            
Update_Mask.on_click(AOI_clicked)

In [34]:
def AOI_clicked(b):
    try:
        with output_widget:
            map_layout.remove_legends()
            global after_filter
            after_filter = ee.Image(toDB(RefinedLee(toNatural(after))))
            global before_filter
            before_filter = ee.Image(toDB(RefinedLee(toNatural(before))))
            global flood_palette
            flood_palette = ['0000FF']
            global flood_class1
            flood_class1 = before_filter.gt(-42).And(after_filter.lt(-26))
            global flood_mask_class1
            flood_mask_class1 = flood_class1.updateMask(flood_class1.eq(1))
            global flood_class2
            flood_class2 = before_filter.gt(-42).And(after_filter.lt(-25))
            global flood_mask_class2
            flood_mask_class2 = flood_class2.updateMask(flood_class2.eq(1))
            global flood_class3
            flood_class3 = before_filter.gt(-42).And(after_filter.lt(-24))
            global flood_mask_class3
            flood_mask_class3 = flood_class3.updateMask(flood_class3.eq(1))

            global flood_class4
            flood_class4 = before_filter.gt(-42).And(after_filter.lt(-23))
            global flood_mask_class4
            flood_mask_class4 = flood_class4.updateMask(flood_class4.eq(1))
                
            global flood_class5
            flood_class5 = before_filter.gt(-42).And(after_filter.lt(-22))
            global flood_mask_class5
            flood_mask_class5 = flood_class5.updateMask(flood_class5.eq(1))
                
            global flood_class6
            flood_class6 = before_filter.gt(-42).And(after_filter.lt(-21))
            global flood_mask_class6
            flood_mask_class6 = flood_class6.updateMask(flood_class6.eq(1))
                
            global flood_class7
            flood_class7 = before_filter.gt(-42).And(after_filter.lt(-20))
            global flood_mask_class7
            flood_mask_class7 = flood_class7.updateMask(flood_class7.eq(1))

            global flood_class8
            flood_class8 = before_filter.gt(-42).And(after_filter.lt(-19))
            global flood_mask_class8
            flood_mask_class8 = flood_class8.updateMask(flood_class8.eq(1))
            global flood_class9
            flood_class9 = before_filter.gt(-42).And(after_filter.lt(-18))
            global flood_mask_class9
            flood_mask_class9 = flood_class9.updateMask(flood_class9.eq(1))
            global flood_class10
            flood_class10 = before_filter.gt(-42).And(after_filter.lt(-17))
            global flood_mask_class10
            flood_mask_class10 = flood_class10.updateMask(flood_class10.eq(1))
                
            global flood_class11
            flood_class11 = before_filter.gt(-42).And(after_filter.lt(-16))
            global flood_mask_class11
            flood_mask_class11 = flood_class11.updateMask(flood_class11.eq(1))
                
            global flood_class12
            flood_class12 = before_filter.gt(-42).And(after_filter.lt(-15))
            global flood_mask_class12
            flood_mask_class12 = flood_class12.updateMask(flood_class12.eq(1))
                
            global flood_class13
            flood_class13 = before_filter.gt(-42).And(after_filter.lt(-14))
            global flood_mask_class13
            flood_mask_class13 = flood_class13.updateMask(flood_class13.eq(1))
                
            global flood_class14
            flood_class14 = before_filter.gt(-42).And(after_filter.lt(-13))
            global flood_mask_class14
            flood_mask_class14 = flood_class14.updateMask(flood_class14.eq(1))
            global flood_class15
            flood_class15 = before_filter.gt(-42).And(after_filter.lt(-12))
            global flood_mask_class15
            flood_mask_class15 = flood_class15.updateMask(flood_class13.eq(1))
                
            global flood_class16
            flood_class16= before_filter.gt(-42).And(after_filter.lt(-11))
            global flood_mask_class16
            flood_mask_class16 = flood_class16.updateMask(flood_class16.eq(1))
#Conditional classes
#Breaking of reclassified imaged into more than one class.
            Rainfall_Amount1=predicted_waterlevel_value88
            User_Rainfall_Amount=(np.array(Rainfall_Amount1).reshape(1,-1))
            predicted_riverlevel=rain_level_regressor.predict(Rainfall_Input)
            print("Expected water rise will be",predicted_riverlevel)
            
            global imp_finalFloodedArea

                    
            if predicted_riverlevel>=0 and predicted_riverlevel<=2:
                map_layout.addLayer(flood_mask_class1,{'palette':flood_palette},"Flood Affected Region")
                
                imp_finalFloodedArea = flood_mask_class1
#                 global plot
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot,x="longitude", y="latitude") 
            elif predicted_riverlevel>=0 and predicted_riverlevel<=4:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot,x="longitude", y="latitude")
                map_layout.addLayer(flood_mask_class2,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class2
            elif predicted_riverlevel>=0 and predicted_riverlevel<=5:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot,x="longitude", y="latitude")
                map_layout.addLayer(flood_mask_class3,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class3
    
            elif predicted_riverlevel>=0 and predicted_riverlevel<=7:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot,x="longitude", y="latitude")
                map_layout.addLayer(flood_mask_class4,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class4
            elif predicted_riverlevel>=0 and predicted_riverlevel<=9:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot,x="longitude", y="latitude")
                map_layout.addLayer(flood_mask_class5,{'palette':flood_palette},"Flood Affected Region")
    
                imp_finalFloodedArea = flood_mask_class5
            elif predicted_riverlevel>=0 and predicted_riverlevel<=12:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
                map_layout.addLayer(flood_mask_class6,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class6
            elif predicted_riverlevel>=0 and predicted_riverlevel<=14:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
#                 slope_palette1 = ['0000FF']
                map_layout.addLayer(flood_mask_class7,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class7
            elif predicted_riverlevel>=0 and predicted_riverlevel<=17:
                map_layout.addLayer(flood_mask_class8,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class8
            elif predicted_riverlevel>=0 and predicted_riverlevel<=19:
                map_layout.addLayer(flood_mask_class9,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class9
            elif predicted_riverlevel>=0 and predicted_riverlevel<=21:
                map_layout.addLayer(flood_mask_class10,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class10
            elif predicted_riverlevel>=0 and predicted_riverlevel<=27:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#               
                map_layout.addLayer(flood_mask_class11,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class11
            elif predicted_riverlevel>=0 and predicted_riverlevel<=28:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
#                 slope_palette1 = ['0000FF']
    
                map_layout.addLayer(flood_mask_class12,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class12
            elif predicted_riverlevel>=0 and predicted_riverlevel<=31:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
#                 slope_palette1 = ['0000FF']
    
                map_layout.addLayer(flood_mask_class13,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class13
            elif predicted_riverlevel>=0 and predicted_riverlevel<=35:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
#                 slope_palette1 = ['0000FF']
    
                map_layout.addLayer(flood_mask_class14,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class14
            elif predicted_riverlevel>=0 and predicted_riverlevel<=3:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
#                 slope_palette1 = ['0000FF']
    
                map_layout.addLayer(flood_mask_class15,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class15

            elif predicted_riverlevel>=0 and predicted_riverlevel<=200:
#                 plot=gdf1.drop(columns=["state","geom","gid"])
#                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
    
                map_layout.addLayer(flood_mask_class16,{'palette':flood_palette},"Flood Affected Region")
                imp_finalFloodedArea = flood_mask_class16
            
            blue_palette = ['f20000']
            jrcGSW = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
            waterBodies = jrcGSW.select('occurrence').clip(boundaries)
            
            vis_waterBodies = {min:0, max:100, "palette": blue_palette}
            map_layout.addLayer(**{
    # mask waterBodies so as to not detect flood in the waterBodies.
    # .divide(100) causes the opacity/transparency of the pixels to
    # be set based on the waterBodies occurrence value.
    
    'ee_object': waterBodies.updateMask(waterBodies.divide(100)),\
    'name': "Permanent water bodies", \
    'vis_params': vis_waterBodies })
#             map_layout.addLayer( waterBodies.updateMask(waterBodies.divide(100)),vis_waterBodies,"Permanent water bodies (BLUE)")
            
            legend_dict = {
                       'Permanent': 'd63000',
                       'Flood Region': '000efb',}
            map_layout.add_legend(legend_title="Legend", legend_dict=legend_dict, position='bottomright',marginright='150px')

    except Exception as e:
        print("An error occurred:", e)

Flood_Flow.on_click(AOI_clicked)


# #estimation of Gross Primary Product.
# def AOI_clicked(b):
#         with output_widget:
#             map_layout.remove_legends()

#             global after_filter
#             after_filter = ee.Image(toDB(RefinedLee(toNatural(after))))
#             global before_filter
#             before_filter = ee.Image(toDB(RefinedLee(toNatural(before))))
#             global flood_palette
#             flood_palette = ['0000FF']
#             global flood_class1
#             flood_class1 = before_filter.gt(-42).And(after_filter.lt(-26))
#             global flood_mask_class1
#             flood_mask_class1 = flood_class1.updateMask(flood_class1.eq(1))
#             global flood_class2
#             flood_class2 = before_filter.gt(-42).And(after_filter.lt(-25))
#             global flood_mask_class2
#             flood_mask_class2 = flood_class2.updateMask(flood_class2.eq(1))
                
#             global flood_class3
#             flood_class3 = before_filter.gt(-42).And(after_filter.lt(-24))
#             global flood_mask_class3
#             flood_mask_class3 = flood_class3.updateMask(flood_class3.eq(1))

#             global flood_class4
#             flood_class4 = before_filter.gt(-42).And(after_filter.lt(-23))
#             global flood_mask_class4
#             flood_mask_class4 = flood_class4.updateMask(flood_class4.eq(1))
                
#             global flood_class5
#             flood_class5 = before_filter.gt(-42).And(after_filter.lt(-22))
#             global flood_mask_class5
#             flood_mask_class5 = flood_class5.updateMask(flood_class5.eq(1))
                
#             global flood_class6
#             flood_class6 = before_filter.gt(-42).And(after_filter.lt(-21))
#             global flood_mask_class6
#             flood_mask_class6 = flood_class6.updateMask(flood_class6.eq(1))
                
#             global flood_class7
#             flood_class7 = before_filter.gt(-42).And(after_filter.lt(-20))
#             global flood_mask_class7
#             flood_mask_class7 = flood_class7.updateMask(flood_class7.eq(1))

#             global flood_class8
#             flood_class8 = before_filter.gt(-42).And(after_filter.lt(-19))
#             global flood_mask_class8
#             flood_mask_class8 = flood_class8.updateMask(flood_class8.eq(1))
#             global flood_class9
#             flood_class9 = before_filter.gt(-42).And(after_filter.lt(-18))
#             global flood_mask_class9
#             flood_mask_class9 = flood_class9.updateMask(flood_class9.eq(1))
#             global flood_class10
#             flood_class10 = before_filter.gt(-42).And(after_filter.lt(-17))
#             global flood_mask_class10
#             flood_mask_class10 = flood_class10.updateMask(flood_class10.eq(1))
                
#             global flood_class11
#             flood_class11 = before_filter.gt(-42).And(after_filter.lt(-16))
#             global flood_mask_class11
#             flood_mask_class11 = flood_class11.updateMask(flood_class11.eq(1))
                
#             global flood_class12
#             flood_class12 = before_filter.gt(-42).And(after_filter.lt(-15))
#             global flood_mask_class12
#             flood_mask_class12 = flood_class12.updateMask(flood_class12.eq(1))
                
#             global flood_class13
#             flood_class13 = before_filter.gt(-42).And(after_filter.lt(-14))
#             global flood_mask_class13
#             flood_mask_class13 = flood_class13.updateMask(flood_class13.eq(1))
                
#             global flood_class14
#             flood_class14 = before_filter.gt(-42).And(after_filter.lt(-13))
#             global flood_mask_class14
#             flood_mask_class14 = flood_class14.updateMask(flood_class14.eq(1))
#             global flood_class15
#             flood_class15 = before_filter.gt(-42).And(after_filter.lt(-12))
#             global flood_mask_class15
#             flood_mask_class15 = flood_class15.updateMask(flood_class13.eq(1))
                
#             global flood_class16
#             flood_class16= before_filter.gt(-42).And(after_filter.lt(-11))
#             global flood_mask_class16
#             flood_mask_class16 = flood_class16.updateMask(flood_class16.eq(1))
# #Conditional classes
# #Breaking of reclassified imaged into more than one class.
#             Rainfall_Amount1=predicted_waterlevel_value88
#             User_Rainfall_Amount=(np.array(Rainfall_Amount1).reshape(1,-1))
#             predicted_riverlevel=rain_level_regressor.predict(Rainfall_Input)
#             print("Expected water rise will be",predicted_riverlevel)
            
#             global imp_finalFloodedArea

                    
#             if predicted_riverlevel>=0 and predicted_riverlevel<=2:
#                 map_layout.addLayer(flood_mask_class1,{'palette':flood_palette},"Flood Affected Region")
                
#                 imp_finalFloodedArea = flood_mask_class1
# #                 global plot
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot,x="longitude", y="latitude") 
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=4:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot,x="longitude", y="latitude")
#                 map_layout.addLayer(flood_mask_class2,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class2
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=5:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot,x="longitude", y="latitude")
#                 map_layout.addLayer(flood_mask_class3,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class3
    
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=7:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot,x="longitude", y="latitude")
#                 map_layout.addLayer(flood_mask_class4,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class4
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=9:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot,x="longitude", y="latitude")
#                 map_layout.addLayer(flood_mask_class5,{'palette':flood_palette},"Flood Affected Region")
    
#                 imp_finalFloodedArea = flood_mask_class5
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=12:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
#                 map_layout.addLayer(flood_mask_class6,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class6
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=14:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
# #                 slope_palette1 = ['0000FF']
#                 map_layout.addLayer(flood_mask_class7,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class7
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=17:
#                 map_layout.addLayer(flood_mask_class8,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class8
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=19:
#                 map_layout.addLayer(flood_mask_class9,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class9
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=21:
#                 map_layout.addLayer(flood_mask_class10,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class10
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=27:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #               
#                 map_layout.addLayer(flood_mask_class11,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class11
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=28:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
# #                 slope_palette1 = ['0000FF']
    
#                 map_layout.addLayer(flood_mask_class12,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class12
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=31:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
# #                 slope_palette1 = ['0000FF']
    
#                 map_layout.addLayer(flood_mask_class13,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class13
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=35:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
# #                 slope_palette1 = ['0000FF']
    
#                 map_layout.addLayer(flood_mask_class14,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class14
#             elif predicted_riverlevel>=0 and predicted_riverlevel<=3:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
# #                 slope_palette1 = ['0000FF']
    
#                 map_layout.addLayer(flood_mask_class15,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class15

#             elif predicted_riverlevel>=0 and predicted_riverlevel<=200:
# #                 plot=gdf1.drop(columns=["state","geom","gid"])
# #                 map_layout.add_points_from_xy(plot, x="longitude", y="latitude")
    
#                 map_layout.addLayer(flood_mask_class16,{'palette':flood_palette},"Flood Affected Region")
#                 imp_finalFloodedArea = flood_mask_class16
            
#             blue_palette = ['f20000']
#             jrcGSW = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
#             waterBodies = jrcGSW.select('occurrence').clip(boundaries)
            
#             vis_waterBodies = {min:0, max:100, "palette": blue_palette}
#             map_layout.addLayer(**{
#     # mask waterBodies so as to not detect flood in the waterBodies.
#     # .divide(100) causes the opacity/transparency of the pixels to
#     # be set based on the waterBodies occurrence value.
    
#     'ee_object': waterBodies.updateMask(waterBodies.divide(100)),\
#     'name': "Permanent water bodies", \
#     'vis_params': vis_waterBodies })
# #             map_layout.addLayer( waterBodies.updateMask(waterBodies.divide(100)),vis_waterBodies,"Permanent water bodies (BLUE)")
            
#             legend_dict = {
#                        'Permanent': 'd63000',
#                        'Flood Region': '000efb',}
#             map_layout.add_legend(legend_title="Legend", legend_dict=legend_dict, position='bottomleft',marginright='150px')

            

# Flood_Flow.on_click(AOI_clicked)

In [35]:
#Creation of an output widgets to ouput outside the map.
user_out = widgets.Output(layout={'border': '1px solid green'})
user_out.clear_output()
user_out

In [36]:
# Click event handler for Estimation of Areas 
def est_yield(b):
    with user_out:
        output_widget.clear_output()
        try:
            global names
            names=["Flood"]
            floodedArea = imp_finalFloodedArea.multiply(ee.Image.pixelArea()).divide(1000000).rename(names)
            floodedAreaSize = floodedArea.reduceRegion( **{
      'reducer': ee.Reducer.sum(),
      'geometry': boundaries.geometry(),
      'scale': 30, 
      'maxPixels': 1e13
            })
            print('Estimated flood Area(KM)', ee.Dictionary(floodedAreaSize).getInfo())  
        except Exception as e:
            print('..')
        else:
            print("Estimated area successfully\n")
cal_area.on_click(est_yield)

In [37]:
#Calculation of mean values from the extracted values from Modis NDVI and Gpp mean values and exporting them to the downloads 
def AOI_clicked(b):
        with user_out:
#             output_widget.clear_output()
           
            try:
        
                  
#Extract Gross primary Product(GPP)
                out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                out_gppCollection_stats = os.path.join(out_dir, 'GPP_Mean_Value.csv')
                
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)

                    
# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
                geemap.zonal_statistics(gppCollection, boundaries, out_gppCollection_stats,statistics_type='MEAN', scale=10)
        
#Extract NDVI
                out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
                NDVI_stats = os.path.join(out_dir, 'NDVI_Mean_Value.csv')
                
                if not os.path.exists(out_dir):
                    os.makedirs(out_dir)

                    
# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
                geemap.zonal_statistics(NDVI, boundaries, NDVI_stats,statistics_type='MEAN', scale=10)
                


               
            except Exception as e:
                print('Please their is a chance that you may skip a step...')
            else:
                print("Successfully run the model....")
                
Export_data.on_click(AOI_clicked)

In [38]:
# import ipywidgets as widgets

# def download_mapped_result(flood_mask_class, region, name):
#     task = batch.Export.image.toDrive(image=flood_mask_class, 
#                                       region=region, 
#                                       description=name, 
#                                       scale=10, 
#                                       max_pixels=1e10)
#     task.start()
#     print(f'Exporting {name}...')

# download_button = widgets.Button(description='Download Result')
# download_button.on_click(lambda x: download_mapped_result(flood_mask_class, region, name))

In [39]:
# def export_data_clicked(b):
#     with user_out:
#         user_out.clear_output()
#         download_classification=check_box_classification.value
#         download_sent_image=check_box_sent_image.value

#         if download_sent_image:
#             print("Click the link below to download Results")
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#             filename = os.path.join(out_dir, 'Mapped.tif')
#             fc = boundaries
#             geometry = fc.geometry()
#             export_image = sentinel_2A.clip(boundaries).unmask()
#             geemap.ee_export_image(
#             sentinel_2A, filename=filename, scale=1000, region=geometry, file_per_band=False)

#         elif download_classification:
#             print("Click the URL below to download classified image")
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#             filename = os.path.join(out_dir, 'Map Results.tif')
#             export_image = landcover.clip(boundaries).unmask()
#             geemap.ee_export_image(landcover, filename=filename, scale=1000, region=boundaries, file_per_band=False)
                
# Export_data.on_click(export_data_clicked)  

In [40]:
def export_data_clicked(b):
    with user_out:
        user_out.clear_output()
        download_classification = check_box_classification.value
        download_sent_image = check_box_sent_image.value
        download_flood_map = check_box_flood_map.value

        if download_sent_image:
            print("Click the link below to download Results")
            out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
            filename = os.path.join(out_dir, 'Mapped.tif')
            fc = boundaries
            geometry = fc.geometry()
            export_image = sentinel_2A.clip(boundaries).unmask()
            geemap.ee_export_image(
                sentinel_2A, filename=filename, scale=1000, region=geometry, file_per_band=False)

        elif download_classification:
            print("Click the URL below to download classified image")
            out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
            filename = os.path.join(out_dir, 'Dynamic LULC.tif')
            fc = boundaries
            geometry = fc.geometry()
            export_image = landcover.clip(boundaries).unmask()
#             export_image = export_image.select(['band1', 'band2', 'band3'])  # Select only the first three bands
            geemap.ee_export_image(
                export_image, filename=filename, scale=30, region=geometry, file_per_band=False, dimensions='256x256')
                 
        elif download_flood_map:
            print("Click the URL below to download the flood map")
            out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
            filename = os.path.join(out_dir, 'Flood Map.tif')
            export_image = flood_mask_class.clip(boundaries).unmask()
            geemap.ee_export_image(
                flood_mask_class, filename=filename, scale=10, region=boundaries, file_per_band=False)

Export_data.on_click(export_data_clicked)


In [41]:
# def export_data_clicked(b):
#     with user_out:
#         user_out.clear_output()
#         download_classification = check_box_classification.value
#         download_sent_image = check_box_sent_image.value
#         download_flood_map = check_box_flood_map.value
#         download_dynamic_lulc = check_box_dynamic_lulc.value  # new checkbox for Dynamic LULC

#         if download_sent_image:
#             print("Click the link below to download Results")
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#             filename = os.path.join(out_dir, 'Mapped.tif')
#             fc = boundaries
#             geometry = fc.geometry()
#             export_image = sentinel_2A.clip(boundaries).unmask()
#             geemap.ee_export_image(
#                 sentinel_2A, filename=filename, scale=1000, region=geometry, file_per_band=False)

#         elif download_classification:
#             print("Click the URL below to download classified image")
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#             filename = os.path.join(out_dir, 'Map Results.tif')
#             export_image = landcover.clip(boundaries).unmask()
#             geemap.ee_export_image(
#                 landcover, filename=filename, scale=1000, region=boundaries, file_per_band=False)

#         elif download_flood_map:
#             print("Click the URL below to download the flood map")
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#             filename = os.path.join(out_dir, 'Flood Map.tif')
#             export_image = flood_mask_class.clip(boundaries).unmask()
#             geemap.ee_export_image(
#                 flood_mask_class, filename=filename, scale=10, region=boundaries, file_per_band=False)
        
#         elif download_dynamic_lulc:  # new block of code for Dynamic LULC
#             print("Click the URL below to download the Dynamic LULC map")
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#             filename = os.path.join(out_dir, 'Dynamic LULC.tif')
#             fc = boundaries
#             geometry = fc.geometry()
#             export_image = dynamic_lulc.clip(boundaries).unmask()
#             geemap.ee_export_image(
#                 export_image, filename=filename, scale=30, region=geometry, file_per_band=False)
            

# Export_data.on_click(export_data_clicked)


In [42]:
# def export_data_clicked(b):
#     with user_out:
#         user_out.clear_output()
#         download_classification = check_box_classification.value
#         download_sent_image = check_box_sent_image.value

#         if download_sent_image:
#             print("Click the link below to download Results")
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#             filename = os.path.join(out_dir, 'Mapped.tif')
#             fc = boundaries
#             geometry = fc.geometry()
#             export_image = sentinel_2A.clip(boundaries).unmask()
#             geemap.ee_export_image(
#                 sentinel_2A,
#                 filename=filename,
#                 scale=1000,
#                 region=geometry,
#                 file_per_band=False)

#         elif download_classification:
#             print("Click the URL below to download classified image")
#             out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#             filename = 'Map_Results'
#             export_image = landcover.clip(boundaries).unmask()
#             task = geemap.ee_export_image_to_drive(
#                 export_image,
#                 description='Map_Results',
#                 folder='GEE_Results',
#                 region=boundaries,
#                 scale=1000
#             )
#             print('Exporting image to Google Drive...')
#             print(task)

# Export_data.on_click(export_data_clicked)


In [10]:
#Reset everything
def reset_me(b):
    with output_widget:
        output_widget.clear_output()
        try:
            map_layout.remove_legends() 
            map_layout.remove_colorbar()
            map_layout.remove_drawn_features()
            map_layout.remove_ee_layer('Dynamic_World')
            map_layout.remove_ee_layer("Classification Training ponts")
            map_layout.remove_ee_layer("Region of Intrest")
            map_layout.remove_ee_layer('Sentinel-2')
            map_layout.remove_ee_layer('classification')
            map_layout.remove_ee_layer('Built-up Regions')
            map_layout.remove_ee_layer('Permanent water bodies (BLUE)')
            map_layout.remove_ee_layer('Permanent water bodies')
            map_layout.remove_ee_layer('boundaries')
            map_layout.remove_ee_layer('building distribution')
            map_layout.remove_ee_layer('LULCC')
            map_layout.remove_ee_layer('Flood Affected Region')

           
        except Exception as e:
            print("Sorry an error just occures..You nay have cleared it already")
        else:
            print("reset successfully....")

Reset_button.on_click(reset_me)

In [57]:
# Below is an additional documentation you can either comment or delete them!!!!!
#Additionally you can add you other if it fit your preferences 

<h3>Documentation & Relevant Datasets:</h3><br>
<h4>Hydrometeorogical Services:</h4>
Data provided:<br>
Main Mission is to observe, archive and understand weather and climate and provide meteorological, hydrological and oceanographic services in support of national needs and international obligations
Learn More: weather Data</a>
<h4>Refined Lee Filter</h4>
The presence of speckle noise in Synthetic Aperture Radar (SAR) images makes the interpretation of the contents difficult, thereby degrading the quality of the image. Therefore an efficient speckle noise removal technique needs to be sought. In this paper, we present a SAR image despeckling method known as Refined Lee Filter (RLF). The method is based on the well-known Lee filter. In this method the K-Nearest Neighbour (KNN) algorithm is employed to adjust the number of neighbour pixels used within the sliding window. Simulation results obtained from the performance evaluation metrics used showed that the proposed method improved the performance of the original Lee filter when they were both used to despeckle standard test images contaminated with speckle noise and SAR images.<br>
Learn More:<a href="https://ieeexplore.ieee.org/document/7334965">Explore More on Refine Lee Filter</a>
<h4>Sentinel-1 SAR GRD: C-band Synthetic Aperture Radar Ground Range Detected, log scaling</h4>
The Sentinel-1 mission provides data from a dual-polarization C-band Synthetic Aperture Radar (SAR) instrument at 5.405GHz (C band). This collection includes the S1 Ground Range Detected (GRD) scenes, processed using the Sentinel-1 Toolbox to generate a calibrated, ortho-corrected product. The collection is updated daily. New assets are ingested within two days after they become available.<br>
Learn More:<a href="https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD">Sentinel Data:COPERNICUS_S1_GRD</a>
<h4>Multivariate regression model from water level and production</h4>
Water management tools are necessary to guarantee the preservation of natural resources while ensuring optimum utilization. Linear regression models are a simple and quick solution for creating prognostic capabilities. Multivariate models show higher precision than univariate models. In the case of Waiwera, implementation of individual production rates is more accurate than applying just the total production rate. A maximum of approximately 1,075 m3/day can be pumped to ensure a water level of at least 0.5 m a.s.l. in the monitoring well. The model should be renewed annually to implement new data and current water level trends to keep the quality.<br>
Learn More:<a href="https://www.sciencedirect.com/science/article/pii/S1876610217336998">Regression Models for water level Modelling</a>
<h4>Hydrosheds</h4>
‍The HydroSHEDS database offers a suite of global digital data layers in support of hydro-ecological research and applications worldwide. Its various hydrographic data products include catchment boundaries, river networks, and lakes at multiple resolutions and scales.<br>
Learn More:<a href="https://www.hydrosheds.org/">Data:HydroSHEDS </a>
<h4>Global Surface Water</h4>
This dataset contains maps of the location and temporal distribution of surface water from 1984 to 2021 and provides statistics on the extent and change of those water surfaces. For more information see the associated journal article: High-resolution mapping of global surface water and its long-term changes (Nature, 2016) and the online Data Users Guide.<br>
Learn More:<a href="https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_4_GlobalSurfaceWater">Data:Global Surface Water Mapping Layer </a>
<h4>Dynamic World:</h4>
Dynamic World is a 10m near-real-time (NRT) Land Use/Land Cover (LULC) dataset that includes class probabilities and label information for nine classes.

Dynamic World predictions are available for the Sentinel-2 L1C collection from 2015-06-27 to present. The revisit frequency of Sentinel-2 is between 2-5 days depending on latitude. Dynamic World predictions are generated for Sentinel-2 L1C images with CLOUDY_PIXEL_PERCENTAGE <= 35%. Predictions are masked to remove clouds and cloud shadows using a combination of S2 Cloud Probability, Cloud Displacement Index, and Directional Distance Transform.
Images in the Dynamic World collection have names matching the individual Sentinel-2 L1C asset names from which they were derived.
Learn More:<a href="https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1">Data:Dynamic World V1 </a>

<center><h5>Powered by Geemap and ipywidgets</h5></center><br>
<center><h5>Author:@ Your Name.. </h5></center><br>
<center><img src="flood-safety_456px.jpg" style="border-radius: 40%"></center>
